# Previsao da Soja em Sorocaba e regiao

Passo 1: Configuração e Bibliotecas<br>
Esta célula prepara o terreno e conecta com o Google Drive e o Google Earth Engine.

In [ ]:
import ee
import geemap
import pandas as pd
import requests
import time
import joblib
from sklearn.ensemble import RandomForestRegressor

# Autenticação (Siga o link e cole o código)
ee.Authenticate()
ee.Initialize(project='agro-proj-01') # Substitua pelo ID do seu projeto se necessário

# Função para calcular NDVI
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

Passo 2: Extração Regional de Dados (Satélite + NASA)<br>
Aqui coletamos o histórico de 2019 a 2024 para todas as cidades. É a parte que demora cerca de 2 minutos.

In [ ]:
# Coordenadas das cidades (Conforme aparecem no seu arquivo IBGE)
cidades_coords = {
    'Sorocaba (SP)': {'lat': -23.5015, 'lon': -47.4521},
    'Itapetininga (SP)': {'lat': -23.5916, 'lon': -48.0530},
    'Itapeva (SP)': {'lat': -23.9819, 'lon': -48.8761},
    'Capão Bonito (SP)': {'lat': -24.0051, 'lon': -48.3496}
}

anos_safra = [2019, 2020, 2021, 2022, 2023, 2024]
dados_regionais = []

for cidade, coord in cidades_coords.items():
    print(f"🛰️ Coletando dados de: {cidade}...")
    geom = ee.Geometry.Point([coord['lon'], coord['lat']]).buffer(5000)

    for ano in anos_safra:
        # NDVI - Médio do pico da safra (Jan/Fev)
        colecao = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterBounds(geom) \
            .filterDate(f'{ano}-01-01', f'{ano}-02-28') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .map(add_ndvi)

        ndvi_valor = colecao.median().reduceRegion(
            reducer=ee.Reducer.mean(), geometry=geom, scale=100
        ).get('NDVI').getInfo()

        # Chuva NASA POWER (Acumulado Outubro a Abril)
        start_nasa, end_nasa = f'{ano-1}1001', f'{ano}0430'
        url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=PRECTOTCORR&community=AG&longitude={coord['lon']}&latitude={coord['lat']}&start={start_nasa}&end={end_nasa}&format=JSON"
        res = requests.get(url)
        chuva_total = sum(res.json()['properties']['parameter']['PRECTOTCORR'].values()) if res.status_code == 200 else 0

        dados_regionais.append({'local': cidade, 'ano': ano, 'ndvi_pico': ndvi_valor, 'chuva_acumulada': chuva_total})
        time.sleep(0.5)

df_regional_hist = pd.DataFrame(dados_regionais)
print("✅ Extração regional finalizada!")

🛰️ Coletando dados de: Sorocaba (SP)...
🛰️ Coletando dados de: Itapetininga (SP)...
🛰️ Coletando dados de: Itapeva (SP)...
🛰️ Coletando dados de: Capão Bonito (SP)...
✅ Extração regional finalizada!


Passo 3: Limpeza do IBGE e Integração Final<br>
Lê o seu CSV e cruza com os dados que acabamos de baixar.

In [ ]:
# Ler CSV do IBGE
caminho_ibge = '/content/drive/MyDrive/Colab/tabela1612.csv'
df_ibge_raw = pd.read_csv(caminho_ibge, sep=None, engine='python', encoding='latin-1', skip_blank_lines=True)
df_ibge_raw.rename(columns={df_ibge_raw.columns[0]: 'local'}, inplace=True)

# Girar tabela (Melt)
anos_str = [str(a) for a in anos_safra]
df_melted = df_ibge_raw.melt(id_vars=['local'], value_vars=anos_str, var_name='ano', value_name='produtividade')
df_melted['ano'] = df_melted['ano'].astype(int)
df_melted['produtividade'] = pd.to_numeric(df_melted['produtividade'], errors='coerce')

# Merge Final
df_final = pd.merge(df_regional_hist, df_melted, on=['local', 'ano']).dropna()
print(f"📊 Dataset pronto com {len(df_final)} linhas.")

📊 Dataset pronto com 18 linhas.


Passo 4: Treinamento e Exportação da IA<br>
Aqui treinamos o modelo e salvamos ele num arquivo.

In [ ]:
X = df_final[['ndvi_pico', 'chuva_acumulada']]
y = df_final['produtividade']

modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X, y)

# Salvar o modelo para usar no Streamlit
joblib.dump(modelo, 'modelo_agro.pkl')
print("🤖 Modelo 'modelo_agro.pkl' salvo com sucesso!")

🤖 Modelo 'modelo_agro.pkl' salvo com sucesso!


Passo 5: Criação do App Streamlit<br>
Célula que gera o arquivo do site.

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

modelo = joblib.load('modelo_agro.pkl')

st.title("🌾 AgroIA: Monitoramento Regional")
st.sidebar.header("Simulador de Safra")

cidade = st.sidebar.selectbox("Cidade", ['Sorocaba (SP)', 'Itapetininga (SP)', 'Itapeva (SP)', 'Capão Bonito (SP)'])
ndvi = st.sidebar.slider("Vigor Vegetativo (NDVI)", 0.4, 0.9, 0.7)
chuva = st.sidebar.number_input("Chuva Acumulada (mm)", 300, 1500, 800)

if st.sidebar.button("Prever Produtividade"):
    pred = modelo.predict(pd.DataFrame([[ndvi, chuva]], columns=['ndvi_pico', 'chuva_acumulada']))[0]
    st.metric(f"Expectativa para {cidade}", f"{pred:.2f} kg/ha")
    st.write("Cálculo realizado com base em Random Forest Regressor treinado com dados ESA/NASA.")

Writing app.py


Passo 6: Lançamento do Site<br>
Rode esta e clique no link gerado. Use o IP que aparecer no console.

In [ ]:
import urllib
print("Endpoint IP:", urllib.request.urlopen('https://ident.me').read().decode('utf8'))
!npx localtunnel --port 8501 & streamlit run app.py

Endpoint IP: 34.150.232.106
/bin/bash: line 1: streamlit: command not found
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://dry-items-sort.loca.lt
